In [1]:
try:
    import cirq
    from iqm.cirq_iqm import Adonis, Apollo, Aphrodite, circuit_from_qasm
    from iqm.cirq_iqm.optimizers import simplify_circuit
except ImportError:
    print('Installing missing dependencies...')
    !pip install --quiet cirq cirq_iqm
    from iqm.cirq_iqm import Adonis, Apollo, circuit_from_qasm
    from iqm.cirq_iqm.optimizers import simplify_circuit
    print('Installation ready')

# The Adonis architecture

Construct an `IQMDevice` instance representing the Adonis architecture, print its qubit connectivity and description

In [2]:
adonis = Adonis()
print(adonis.__doc__)
print(adonis.metadata.gateset)
print(adonis.qubits)

IQM's five-qubit transmon device.

    The qubits are connected thus::

            QB1
             |
      QB2 - QB3 - QB4
             |
            QB5

    where the lines denote which qubit pairs can be subject to two-qubit gates.

    Adonis has native PhasedXPowGate, XPowGate, and YPowGate gates. The two-qubit gate CZ is
    native, as well. The qubits can be measured simultaneously or separately any number of times.
    
Gateset: 
Type GateFamily: cirq.ops.phased_x_gate.PhasedXPowGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.phased_x_gate.PhasedXPowGate)`

Type GateFamily: cirq.ops.common_gates.XPowGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.common_gates.XPowGate)`

Type GateFamily: cirq.ops.common_gates.YPowGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.common_gates.YPowGate)`

Type GateFamily: cirq.ops.measurement_gate.MeasurementGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.measurement_

# Creating a quantum circuit

Create a quantum circuit and insert native gates

In [3]:
a, b, c = adonis.qubits[:3]
circuit = cirq.Circuit()
circuit.append(cirq.X(a))
circuit.append(cirq.PhasedXPowGate(phase_exponent=0.3, exponent=0.5)(c))
circuit.append(cirq.CZ(a, c))
circuit.append(cirq.YPowGate(exponent=1.1)(c))
print(circuit)

QB1 (d=2): ───X──────────────@────────────
                             │
QB3 (d=2): ───PhX(0.3)^0.5───@───Y^-0.9───


Insert non-native gates and decompose them into native ones by calling `decompose_circuit`

In [4]:
circuit.append(cirq.ZZPowGate(exponent=0.2, global_shift=-0.5)(a, c))
circuit.append(cirq.HPowGate(exponent=-0.4)(a))
circuit = adonis.decompose_circuit(circuit)
print(circuit)

QB1 (d=2): ───X──────────────@────────────────────────@────────────@───Y^0.25─────X^-0.4───Y^-0.25───
                             │                        │            │
QB3 (d=2): ───PhX(0.3)^0.5───@───Y^-0.9───Ry(-0.5π)───@───X^-0.2───@───Ry(0.5π)──────────────────────


# Optimizing a quantum circuit

Use the `simplify_circuit` method to run a sequence of optimization passes on a circuit

In [5]:
circuit = cirq.Circuit([
    cirq.H(a),
    cirq.CNOT(a, c),
    cirq.measure(a, c, key='result')])
print(circuit)

QB1 (d=2): ───H───@───M('result')───
                  │   │
QB3 (d=2): ───────X───M─────────────


In [6]:
circuit = adonis.decompose_circuit(circuit)
circuit = simplify_circuit(circuit)
print(circuit)

QB1 (d=2): ───PhX(-0.5)^0.5───@──────────────────M('result')───
                              │                  │
QB3 (d=2): ───PhX(-0.5)^0.5───@───PhX(0.5)^0.5───M─────────────


# Simulating a quantum circuit

Circuits that contain IQM-native gates can be simulated using the standard Cirq simulators

In [7]:
sim = cirq.Simulator()
samples = sim.run(circuit, repetitions=100)

print('Samples:')
print(samples.histogram(key='result'))
print('\nState before the measurement:')
result = sim.simulate(circuit[:-1])
print(result)

Samples:
Counter({3: 53, 0: 47})

State before the measurement:
measurements: (no measurements)

qubits: (cirq.NamedQid('QB1', dimension=2), cirq.NamedQid('QB3', dimension=2))
output vector: (-0.5+0.5j)|00⟩ + (0.5-0.5j)|11⟩

phase:
output vector: |⟩


Note that the above output vector represents the state before the measurement in the optimized circuit, not the original one, which would have the same phase for both terms. `simplify_circuit` has eliminated a `ZPowGate` which has no effect on the measurement.

---

# Creating a quantum circuit from an OpenQASM 2.0 program


In [8]:
qasm_program = """
    OPENQASM 2.0;
    include "qelib1.inc";
    qreg q[3];
    creg meas[3];
    rx(1.7) q[1];
    h q[0];
    cx q[1], q[2];
"""
circuit = circuit_from_qasm(qasm_program)
print(circuit)

q_0: ───H────────────────

q_1: ───Rx(0.541π)───@───
                     │
q_2: ────────────────X───


Decompose the circuit for the Adonis architecture

In [9]:
decomposed = adonis.decompose_circuit(circuit)
print(decomposed)

q_0: ───Y^0.5────────X───────────

q_1: ───Rx(0.541π)───@───────────
                     │
q_2: ───Y^-0.5───────@───Y^0.5───


Map the circuit qubits to device qubits manually

In [10]:
qubit_mapping = {cirq.NamedQubit(k): v for k, v in {'q_0': a, 'q_1': b, 'q_2': c}.items()}
mapped = decomposed.transform_qubits(qubit_mapping)
print(mapped)

QB1 (d=2): ───Y^0.5────────X───────────

QB2 (d=2): ───Rx(0.541π)───@───────────
                           │
QB3 (d=2): ───Y^-0.5───────@───Y^0.5───


or automatically

In [11]:
mapped, _, _ = adonis.route_circuit(decomposed)
print(mapped)

                           ┌──┐
QB1 (d=2): ───Rx(0.541π)─────@────────────
                             │
QB2 (d=2): ───Y^0.5─────────X┼────────────
                             │
QB3 (d=2): ───Y^-0.5─────────@────Y^0.5───
                           └──┘


See the `examples` directory for more examples.

---

# The Apollo architecture

This section describes the same workflow using the 20-qubit Apollo architecture.

Construct an `IQMDevice` instance representing the Apollo architecture, print its qubit connectivity and description

In [12]:
apollo = Apollo()
print(apollo.__doc__)
print(apollo.metadata.gateset)
print(apollo.qubits)

IQM's twenty-qubit transmon device.

    The qubits are connected thus::

            QB20  QB17
            /  \  /  \
         QB19  QB16  QB12
         /  \  /  \  /  \
      QB18  QB15  QB11  QB7
         \  /  \  /  \  /
         QB14  QB10  QB6
         /  \  /  \  /
      QB13  QB9   QB5
         \  /  \  /  \
         QB8   QB4   QB2
            \  /  \  /
            QB3   QB1

    where the lines denote which qubit pairs can be subject to two-qubit gates.

    Apollo has native PhasedXPowGate, XPowGate, and YPowGate gates. The two-qubit gate CZ is
    native, as well. The qubits can be measured simultaneously or separately any number of times.
    
Gateset: 
Type GateFamily: cirq.ops.phased_x_gate.PhasedXPowGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.phased_x_gate.PhasedXPowGate)`

Type GateFamily: cirq.ops.common_gates.XPowGate
Accepts `cirq.Gate` instances `g` s.t. `isinstance(g, cirq.ops.common_gates.XPowGate)`

Type GateFamily: cirq.ops.common_gate

Create a circuit and insert native gates

In [13]:
a, b, c, d, e, f = apollo.qubits[:6]
circuit = cirq.Circuit()
circuit.append(cirq.CZ(a, b))
circuit.append(cirq.CZ(a, c))
circuit.append(cirq.YPowGate(exponent=1.1)(d))
circuit.append(cirq.YPowGate(exponent=1.1)(e))
circuit.append(cirq.PhasedXPowGate(phase_exponent=0.3, exponent=0.5)(f))
print(circuit)

QB1 (d=2): ───@──────────────@───
              │              │
QB2 (d=2): ───@──────────────┼───
                             │
QB3 (d=2): ──────────────────@───

QB4 (d=2): ───Y^-0.9─────────────

QB5 (d=2): ───Y^-0.9─────────────

QB6 (d=2): ───PhX(0.3)^0.5───────


Insert non-native gates and decompose the circuit so that it contains native gates only

In [14]:
circuit.append(cirq.ZZPowGate(exponent=0.2, global_shift=-0.5)(d, e))
circuit.append(cirq.HPowGate(exponent=-0.4)(f))
circuit = apollo.decompose_circuit(circuit)
print(circuit)

QB1 (d=2): ───@──────────────@─────────────────────────────────────────────
              │              │
QB2 (d=2): ───@──────────────┼─────────────────────────────────────────────
                             │
QB3 (d=2): ──────────────────@─────────────────────────────────────────────

QB4 (d=2): ───Y^-0.9─────────────────────@──────────────────@──────────────
                                         │                  │
QB5 (d=2): ───Y^-0.9─────────Ry(-0.5π)───@────────X^-0.2────@───Ry(0.5π)───

QB6 (d=2): ───PhX(0.3)^0.5───Y^0.25──────X^-0.4───Y^-0.25──────────────────


Use the `simplify_circuit` function to run a sequence of optimization passes on a circuit

In [15]:
circuit = apollo.decompose_circuit(circuit)
circuit = simplify_circuit(circuit)
print(circuit)

QB1 (d=2): ───@─────────────────────────────@───────────────────────────────────────
              │                             │
QB2 (d=2): ───@─────────────────────────────┼───────────────────────────────────────
                                            │
QB3 (d=2): ─────────────────────────────────@───────────────────────────────────────

QB4 (d=2): ───PhX(-0.5)^0.9─────────────────────@────────────────@──────────────────
                                                │                │
QB5 (d=2): ───PhX(0.5)^0.6──────────────────────@───PhX(1)^0.2───@───PhX(0.5)^0.5───

QB6 (d=2): ───PhX(0.453)^0.264───Z^-0.395───────────────────────────────────────────


In [16]:
circuit.append(cirq.measure(a, f, key='result'))

sim = cirq.Simulator()
samples = sim.run(circuit, repetitions=100)

print('Samples:')
print(samples.histogram(key='result'))
print('\nState before the measurement:')
result = sim.simulate(circuit[:-1])
print(result)

Samples:
Counter({0: 82, 1: 18})

State before the measurement:
measurements: result=00

qubits: (cirq.NamedQid('QB1', dimension=2),)
output vector: (0.915+0.403j)|0⟩

qubits: (cirq.NamedQid('QB2', dimension=2), cirq.NamedQid('QB3', dimension=2))
output vector: |00⟩

qubits: (cirq.NamedQid('QB4', dimension=2), cirq.NamedQid('QB5', dimension=2))
output vector: (-0.096+0.005j)|00⟩ + (-0.12+0.029j)|01⟩ + (0.38-0.471j)|10⟩ + (0.596-0.505j)|11⟩

qubits: (cirq.NamedQid('QB6', dimension=2),)
output vector: |0⟩

phase:
output vector: |⟩


---

# The Aphrodite device

This section describes the same workflow for using the 54 qubit Aphrodite architecture.

Construct an `IQMDevice` instance representing the Aphrodite architecture, print its qubit connectivity and description

In [17]:
aphrodite = Aphrodite()
print(aphrodite.__doc__)
print(aphrodite.metadata.gateset)
print(aphrodite.qubits)

IQM's 54-qubit transmon device.

    The qubits are connected thus::

          QB54  QB51  QB46  QB39
          /  \  /  \  /  \  /  \
       QB53  QB50  QB45  QB38  QB31
       /  \  /  \  /  \  /  \  /
    QB52  QB49  QB44  QB37  QB30
       \  /  \  /  \  /  \  /  \
       QB48  QB43  QB36  QB29  QB22
       /  \  /  \  /  \  /  \  /  \
    QB47  QB42  QB35  QB28  QB21  QB14
       \  /  \  /  \  /  \  /  \  /
       QB41  QB34  QB27  QB20  QB13
       /  \  /  \  /  \  /  \  /  \
    QB40  QB33  QB26  QB19  QB12  QB7
       \  /  \  /  \  /  \  /  \  /
       QB32  QB25  QB18  QB11  QB6
          \  /  \  /  \  /  \  /  \
          QB24  QB17  QB10  QB5   QB2
          /  \  /  \  /  \  /  \  /
       QB23  QB16  QB9   QB4   QB1
          \  /  \  /  \  /
          QB15  QB8   QB3

    where the lines denote which qubit pairs can be subject to two-qubit gates.

    Aphrodite has native PhasedXPowGate, XPowGate, and YPowGate gates. The two-qubit gate CZ is
    native, as well. The 

Create a simple circuit, for example a GHZ state

In [18]:
circuit = cirq.Circuit()
circuit.append(cirq.H(aphrodite.qubits[0]))
for ctrl, targ in zip(aphrodite.qubits[:5], aphrodite.qubits[1:6]):
    circuit.append(cirq.CX(ctrl, targ))
circuit.append(cirq.measure(aphrodite.qubits[:6], key='result'))
print(circuit)

QB1 (d=2): ───H───@───────────────────M('result')───
                  │                   │
QB2 (d=2): ───────X───@───────────────M─────────────
                      │               │
QB3 (d=2): ───────────X───@───────────M─────────────
                          │           │
QB4 (d=2): ───────────────X───@───────M─────────────
                              │       │
QB5 (d=2): ───────────────────X───@───M─────────────
                                  │   │
QB6 (d=2): ───────────────────────X───M─────────────


Transpile the circuit to fit the architecture:
    - Turn the circuit into native gates
    - Simplify the circuit to remove redundant gates
    - Route the circuit to fit the qubit connectivity
    - Decompose the circuit again because routing can add non-native SWAP gates
    - Simplify the circuit again because some components of the decomposed SWAP can cancel out

In [19]:
native_circuit = aphrodite.decompose_circuit(circuit)
simplified_circuit = simplify_circuit(native_circuit)
routed_circuit, _, _ = aphrodite.route_circuit(simplified_circuit)
native_routed_circuit = aphrodite.decompose_circuit(routed_circuit)
transpiled_circuit = simplify_circuit(native_routed_circuit)
print(transpiled_circuit)

QB5 (d=2): ────PhX(-0.5)^0.5──────────────────────@───PhX(0.5)^0.5───@────────────────────────────────────────────────────────M─────────────
                                                  │                  │                                                        │
QB6 (d=2): ────PhX(-0.5)^0.5───@───PhX(0.5)^0.5───@──────────────────┼────────────────────────────────────────────────────────M─────────────
                               │                                     │                                                        │
QB11 (d=2): ───PhX(-0.5)^0.5───┼─────────────────────────────────────@───PhX(0.5)^0.5───@─────────────────────────────────────M─────────────
                               │                                                        │                                     │
QB12 (d=2): ───PhX(-0.5)^0.5───@────────────────────────────────────────────────────────┼─────────────────────────────────────M('result')───
                                                    

Run the simulator

In [20]:
sim = cirq.Simulator()
samples = sim.run(transpiled_circuit, repetitions=100)

print('Samples:')
print(samples.histogram(key='result'))
print('\nState before the measurement:')
result = sim.simulate(transpiled_circuit[:-1])
print(result)

Samples:
Counter({0: 51, 63: 49})

State before the measurement:
measurements: (no measurements)

qubits: (cirq.NamedQid('QB12', dimension=2), cirq.NamedQid('QB6', dimension=2), cirq.NamedQid('QB5', dimension=2), cirq.NamedQid('QB11', dimension=2), cirq.NamedQid('QB19', dimension=2), cirq.NamedQid('QB27', dimension=2))
output vector: (-0.5+0.5j)|000000⟩ + (0.5-0.5j)|111111⟩

phase:
output vector: |⟩
